In [1]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
from windHelper import WindHelper
from factorModel.factorValidator import *
from datetime import *

In [2]:
beginDate = datetime(2016, 1, 1)
endDate = datetime(2017, 9, 9)
para = "close"
codeList=["T.CFE","RB.SHF"]
df = WindHelper.getMultiTimeSeriesDataFrame(codeList=codeList, beginDate=beginDate, endDate=endDate, para=para)
df = df.dropna()
df
asset_series = df[(codeList[0]).lower() + "_close"]
factor_series = df[codeList[1].lower() + "_close"]
df.head()

Welcome to use Wind Quant API for Python (WindPy)!
You can use w.menu to help yourself to create commands(WSD,WSS,WST,WSI,WSQ,...)!

COPYRIGHT (C) 2017 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


,rb.shf_close,t.cfe_close
trade_date,,
2016-01-04,1777.0,99.865
2016-01-05,1789.0,99.595
2016-01-06,1775.0,100.020
2016-01-07,1773.0,99.865
2016-01-08,1759.0,100.240


# 观察国债期货和螺纹钢的关系

In [5]:
import pandas_highcharts
from pandas_highcharts.display import display_charts
import matplotlib.pyplot as plt
display_charts(df, chart_type="stock",secondary_y=df.columns[1], title="CRB测试")

# 夏普比率校验

In [32]:
config = {"relation": NEGATIVE, "type": UP, "his_count": 30,"std_count":0, "period": 5}

In [33]:
report = sharp_ratio_test(asset_series=asset_series, 
                             factor_series=factor_series, 
                             event_name="e001", 
                             config=config)

In [34]:
import pandas_highcharts
from pandas_highcharts.display import display_charts
import matplotlib.pyplot as plte

print report['sp']
print report['excess_sp']
display_charts(report["ts"][["base","value"]], chart_type="stock", title="夏普比率测试",)

-1.14650710476
-0.00502738272195


# 信号PAE校验

重新检视配置文件

In [28]:
config = {"period": 22, "relation": NEGATIVE, "type":UP, "his_count": 60,"std_count":2}

In [29]:
report = perform_after_event(asset_series=asset_series, 
                             factor_series=factor_series, 
                             event_name="e001", 
                             config=config,
                             asset_price=True)

In [30]:
%matplotlib inline
report["ts"]["ret"]

2006-04-10   -0.009853
2006-05-17   -0.012273
2007-03-08    0.001237
2008-03-05    0.008529
2009-08-10    0.003192
2009-11-18    0.002599
2010-08-06    0.003625
2011-01-20   -0.001990
2014-02-27   -0.008854
2016-03-23   -0.021993
2016-04-25    0.016996
2016-12-09   -0.013456
Name: ret, dtype: float64

In [ ]:
len(report["ts"]["ret"][report["ts"]["ret"]<0])*1.0/len(report["ts"]["ret"])